<a href="https://colab.research.google.com/github/superklex/Hamoye-externship/blob/master/Classification_Quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
data = [[355, 1480], [45, 120]]
conf_matrix = pd.DataFrame(data, index=['Predicted: Spam', 'Predicted: Not Spam'],columns=['Actual: Spam', 'Actual: Not Spam'])

conf_matrix
TP = conf_matrix.iloc[0][0]
FP = conf_matrix.iloc[0][1]
TN = conf_matrix.iloc[1][1]
FN = conf_matrix.iloc[1][0]

In [2]:
#f1= 2*((precision*recall)/(precision+recall))
precision = TP/(TP + FP)
recall = TP/(TP + FN)
f1= 2*((precision*recall)/(precision+recall))
f1


0.3176733780760626

In [3]:
import pandas as pd
import numpy as np
import sklearn

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/superklex/my_datasets/master/Data_for_UCI_named.csv")

In [5]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [6]:
df.isnull().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [7]:
df.dtypes

tau1     float64
tau2     float64
tau3     float64
tau4     float64
p1       float64
p2       float64
p3       float64
p4       float64
g1       float64
g2       float64
g3       float64
g4       float64
stab     float64
stabf     object
dtype: object

In [8]:
#dropping the stab column
df = df.drop("stab", axis =1)

In [9]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
df.stabf = label.fit(df.stabf).transform(df.stabf)

In [10]:
df.stabf

0       1
1       0
2       1
3       1
4       1
       ..
9995    1
9996    0
9997    0
9998    1
9999    1
Name: stabf, Length: 10000, dtype: int64

In [11]:
#importing a scaler for the data
from sklearn.preprocessing import StandardScaler
scaler =StandardScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)

In [12]:
#getting target and features
x = scaled_df.drop("stabf", axis = 1)
y = df.stabf

In [13]:
#splitting
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)

In [14]:
# Importing Random Forest Classifier, setting the random state = 1
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state = 1)

In [15]:
clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [16]:
# Predicting with the test data
clf_pred = clf.predict(x_test)

In [17]:
#checking the accuracy
from sklearn.metrics import accuracy_score
clf_acc = accuracy_score(clf_pred, y_test)
print(clf_acc)

0.929


In [18]:
import xgboost as xgb
model = xgb.XGBClassifier(random_state = 1)

# Fit the model
model.fit(x_train, y_train)

# Predicting with the test data
model_pred = model.predict(x_test)

#checking the accuracy
model_acc = accuracy_score(model_pred, y_test)
print(model_acc)

0.9195


In [19]:
import lightgbm as gbm
lgbm = gbm.LGBMClassifier(random_state = 1)

# Fit the model
lgbm.fit(x_train, y_train)

# Predicting with the test data
lgbm_pred = lgbm.predict(x_test)

# checking the accuracy
lgbm_acc = accuracy_score(lgbm_pred, y_test)
print(lgbm_acc)

0.9475


In [20]:
from sklearn.ensemble import ExtraTreesClassifier
extra = ExtraTreesClassifier()

# Hyper-parameters
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]

# Parameter grid
param_grid = {'n_estimators': n_estimators, 'min_samples_leaf': min_samples_leaf, 'min_samples_split': min_samples_split, 'max_features': max_features}


In [21]:
# Importing Random Search CV
from sklearn.model_selection import RandomizedSearchCV

# Random Search for hyper-parameter optimization
rscv = RandomizedSearchCV(extra, param_grid, cv = 5, n_iter = 10, scoring = 'accuracy', n_jobs = -1, verbose = 1, random_state = 1)

# Fit the model
rscv.fit(x_train, y_train)

# best estimators
print(rscv.best_estimator_)
print(rscv.best_estimator_.min_samples_split)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.6min finished


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features=None,
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=8, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=1000,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)
2


In [22]:
#new optimal model

extclass = ExtraTreesClassifier(max_features = None, min_samples_leaf = 8, n_estimators = 1000, min_samples_split = 2, random_state = 1)

# Fit the model
extclass.fit(x_train, y_train)

# Predicting with the test data
extclass_pred = extclass.predict(x_test)

# Obtain the accuracy
extclass_acc = accuracy_score(extclass_pred, y_test)
print(extclass_acc)

# Taking the initial Extra Trees Classifier with no hyper-parameter tuning
# Fit the model
extra.fit(x_train, y_train)

# Predict on the test data
extra_pred = extra.predict(x_test)

# Obtain the accuracy
extra_acc = accuracy_score(extra_pred, y_test)
print(extra_acc)



0.927
0.9245


In [23]:
extclass_acc < extra_acc

False

In [24]:
# Function to obtain the important features
def get_weights_df(model, feat, col_name):
  weights = pd.Series(model.feature_importances_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

extra.fit(x_train, y_train)

model_weights = get_weights_df(extra, x_train, 'Feature_Importance')
model_weights.nsmallest

<bound method DataFrame.nsmallest of    Features  Feature_Importance
0        p1            0.040283
1        p2            0.040510
2        p4            0.040557
3        p3            0.041022
4        g1            0.088812
5        g4            0.093256
6        g2            0.094524
7        g3            0.096135
8      tau3            0.115091
9      tau4            0.115109
10     tau1            0.116725
11     tau2            0.117978>

In [25]:
model_weights

,Features,Feature_Importance
0,p1,0.040283
1,p2,0.040510
2,p4,0.040557
3,p3,0.041022
4,g1,0.088812
5,g4,0.093256
6,g2,0.094524
7,g3,0.096135
8,tau3,0.115091
9,tau4,0.115109
